# Addition with T5

In [1]:
%%capture
!pip install pytorch-lightning
!pip install transformers

In [1]:
import os
import glob
import json
import random
from typing import List

import torch
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
def compute_exact_match(predicted_answer, correct_answer) -> bool:
    """
    Return True if the predicted answer matches the correct answer,
    return False otherwise.
    """
    predicted_answer = predicted_answer.strip().lower()
    correct_answer = correct_answer.strip().lower()
    return predicted_answer == correct_answer

Create a dictionary with the hyperparameter configuration.

In [3]:
hparams = dict()
hparams['model_name_or_path'] = 't5-base'
hparams['max_seq_length'] = 512
hparams['optimizer'] = 'AdamW'
hparams['lr'] = 3e-4
hparams['weight_decay'] = 5e-5

Create a custom class for finetuning T5.

In [4]:
class T5Finetuner(pl.LightningModule):

    def __init__(self, hparams, train_dataloader, val_dataloader):
        super(T5Finetuner, self).__init__()

        for key in hparams.keys():
            self.hparams[key]=hparams[key]

        self.tokenizer = AutoTokenizer.from_pretrained(self.hparams['model_name_or_path'])
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.hparams['model_name_or_path'])

        self._train_dataloader = train_dataloader
        self._val_dataloader = val_dataloader

        self.validation_step_outputs = []

    def prepare_batch(self, questions: List[str], answers: List[str]) -> List[str]:

        input_dict = self.tokenizer.batch_encode_plus(
            list(questions), padding=True, truncation=False, return_tensors='pt')

        labels = self.tokenizer.batch_encode_plus(
            list(answers), padding=True, truncation=False, return_tensors='pt')['input_ids']

        assert input_dict['input_ids'].shape[1] < self.hparams['max_seq_length']
        assert labels.shape[1] < self.hparams['max_seq_length']

        input_ids = input_dict['input_ids'].to(self.model.device)
        attention_mask = input_dict['attention_mask'].to(self.model.device)
        labels = labels.to(self.model.device)

        return input_ids, attention_mask, labels

    def forward(self, **kwargs):
        return self.model(**kwargs)

    def training_step(self, batch, batch_nb):
        questions, correct_answers = batch

        input_ids, attention_mask, labels = self.prepare_batch(
            questions=questions, answers=correct_answers)

        loss = self.model(input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels)[0]

        tensorboard_logs = {'train_loss': loss}

        return {'loss': loss, 'log': tensorboard_logs}

    def inference_step(self, batch, batch_nb: int):
        questions, correct_answers = batch

        input_ids, attention_mask, _ = self.prepare_batch(
            questions=questions, answers=correct_answers)

        batch_outputs = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            do_sample=False,
            max_length=self.hparams['max_seq_length'])

        predicted_answers = [
            self.tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for output in batch_outputs]

        exact_matches = [
            compute_exact_match(predicted_answer=predicted_answer, correct_answer=correct_answer)
            for predicted_answer, correct_answer in zip(predicted_answers, correct_answers)]

        # Log every power of two.
        if batch_nb & (batch_nb - 1) == 0:
            print('\nQuestion:', questions[0])
            print('Correct:  ', correct_answers[0])
            print('Predicted:', predicted_answers[0].encode('utf-8'))
            print('Exact?', exact_matches[0])

        metrics = {'exact_matches': exact_matches}
        return metrics

    def validation_step(self, batch, batch_nb):
        res = self.inference_step(batch, batch_nb)
        self.validation_step_outputs.append(res)
        return res

    def test_step(self, batch, batch_nb):
        res = self.inference_step(batch, batch_nb)
        self.test_step_outputs.append(res)
        return res

    def on_validation_epoch_end(self):
        exact_matches = []
        for x in self.validation_step_outputs:
            exact_matches.extend(x['exact_matches'])
        exact_match = sum(exact_matches) / len(exact_matches)

        metrics = {'val_exact_match': exact_match}
        self.log('val_exact_match', exact_match)

        output = metrics.copy()
        output['progress_bar'] = metrics
        self.validation_step_outputs.clear()
        print("Accuracy / Exact Match:", exact_match)

        return output

    def train_dataloader(self):
        return self._train_dataloader

    def val_dataloader(self):
        return self._val_dataloader

    def get_optimizer(self):
        optimizer_name = self.hparams['optimizer']
        lr = self.hparams['lr']
        weight_decay = self.hparams['weight_decay']

        optimizer = getattr(torch.optim, optimizer_name)

        # Prepare optimizer 
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {
                'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
                'weight_decay': weight_decay,
            },
            {
                'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)],
                'weight_decay': 0.0
            },
        ]
        optimizer = optimizer(optimizer_grouped_parameters, lr=lr, weight_decay=weight_decay)

        print(f'=> Using {optimizer_name} optimizer')

        return [optimizer]

    def configure_optimizers(self):
        optimizer = self.get_optimizer()
        return optimizer

Create a class for generating addition problem datasets. The numbers are generated randomly and have a roughly equal proportion of d-digit numbers, where d ∈ [min_digits, max_digits].

In [5]:
class MyDataset(Dataset):
    def __init__(self, n_examples, min_digits, max_digits):

        self.examples = []
        for _ in range(n_examples):
            example = []
            for _ in range(2):
                max_digits_i = random.randint(min_digits, max_digits)
                min_number = int((max_digits_i - 1) * '9') + 1
                max_number = int(max_digits_i * '9')
                example.append(random.randint(min_number, max_number))
            self.examples.append(example)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        first_term, second_term = self.examples[idx]

        result = first_term + second_term

        first_term = self.convert_number(first_term)
        second_term = self.convert_number(second_term)
        answer = self.convert_number(result)

        return f'What is {first_term} plus {second_term}?', answer

    def convert_number(self, number) -> str:
        number = str(number)
        signal = None
        if number[0] == '-':
            signal = '-'
            number = number[1:]

        output = []
        len_number = len(number) - 1
        for i, digit in enumerate(number):
            output.append(digit)
            output.append('10e' + str(len_number - i))

        if signal:
            output.append(signal)

        return ' '.join(output)

Generate 
- the training dataset, 
- the evaluation dataset to monitor the exact match / accuracy throughout the training process.

In [6]:
train_dataset = MyDataset(
    n_examples=10000, 
    min_digits=2, 
    max_digits=30
)

val_dataset = MyDataset(
    n_examples=200, 
    min_digits=2, 
    max_digits=30
)

Create dataloaders for the three datasets.

In [7]:
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=16,
    shuffle=True, 
    num_workers=2
)

val_dataloader = DataLoader(
    val_dataset, 
    batch_size=16,
    shuffle=False, 
    num_workers=2
)

Finetune T5 for as many epochs as possible. Every 10 epochs, accuracy will be calculated on the validation dataset, and the model will be saved as a checkpoint.

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath='/content',
    filename='{epoch}-{val_exact_match:.4f}',
    verbose=False, save_last=False, 
    save_top_k=1, mode='max', 
    monitor='val_exact_match',
    save_weights_only=False, 
    every_n_epochs=10
)

trainer = pl.Trainer(
    gradient_clip_val=1.0, 
    accumulate_grad_batches=32, 
    check_val_every_n_epoch=1,
    max_epochs=40,
    callbacks=[checkpoint_callback]
)

model = T5Finetuner(
    hparams=hparams,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader
)

trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this

=> Using AdamW optimizer


Sanity Checking: 0it [00:00, ?it/s]


Question: What is 6 10e21 4 10e20 6 10e19 9 10e18 0 10e17 4 10e16 6 10e15 3 10e14 1 10e13 0 10e12 0 10e11 6 10e10 8 10e9 3 10e8 5 10e7 7 10e6 1 10e5 2 10e4 6 10e3 4 10e2 4 10e1 1 10e0 plus 7 10e22 6 10e21 7 10e20 5 10e19 2 10e18 8 10e17 5 10e16 9 10e15 9 10e14 6 10e13 3 10e12 1 10e11 2 10e10 2 10e9 3 10e8 8 10e7 0 10e6 0 10e5 9 10e4 9 10e3 6 10e2 5 10e1 7 10e0?
Correct:   8 10e22 3 10e21 2 10e20 2 10e19 1 10e18 9 10e17 0 10e16 6 10e15 2 10e14 7 10e13 3 10e12 1 10e11 9 10e10 0 10e9 7 10e8 3 10e7 7 10e6 2 10e5 2 10e4 6 10e3 0 10e2 9 10e1 8 10e0
Predicted: b'6 0 10e5 9 10e3 6 10e2 5 10e1 7 10e0 plus 7 10e22 6 10e21 4 10e20 6 10e19 9 10e18 0 10e17 4 10e16 6 10e15 3 10e14 1 10e13 0 10e12 0 10e11 6 10e10 8 10e9 3 10e8 5 10e7 7 10e6 1 10e5 2 10e4 6 10e3 4 10e2 4 10'
Exact? False

Question: What is 6 10e5 6 10e4 1 10e3 7 10e2 6 10e1 1 10e0 plus 6 10e16 9 10e15 5 10e14 8 10e13 3 10e12 0 10e11 5 10e10 2 10e9 4 10e8 5 10e7 1 10e6 8 10e5 0 10e4 4 10e3 6 10e2 3 10e1 8 10e0?
Correct:   6 10e16 9 10

Training: 0it [00:00, ?it/s]

...


Question: What is 6 10e21 4 10e20 6 10e19 9 10e18 0 10e17 4 10e16 6 10e15 3 10e14 1 10e13 0 10e12 0 10e11 6 10e10 8 10e9 3 10e8 5 10e7 7 10e6 1 10e5 2 10e4 6 10e3 4 10e2 4 10e1 1 10e0 plus 7 10e22 6 10e21 7 10e20 5 10e19 2 10e18 8 10e17 5 10e16 9 10e15 9 10e14 6 10e13 3 10e12 1 10e11 2 10e10 2 10e9 3 10e8 8 10e7 0 10e6 0 10e5 9 10e4 9 10e3 6 10e2 5 10e1 7 10e0?
Correct:   8 10e22 3 10e21 2 10e20 2 10e19 1 10e18 9 10e17 0 10e16 6 10e15 2 10e14 7 10e13 3 10e12 1 10e11 9 10e10 0 10e9 7 10e8 3 10e7 7 10e6 2 10e5 2 10e4 6 10e3 0 10e2 9 10e1 8 10e0
Predicted: b'8 10e22 3 10e21 2 10e20 2 10e19 1 10e18 9 10e17 0 10e16 6 10e15 2 10e14 7 10e13 3 10e12 1 10e11 9 10e10 0 10e9 7 10e8 3 10e7 7 10e6 2 10e5 2 10e4 6 10e3 0 10e2 9 10e1 8 10e0'
Exact? True

Question: What is 6 10e5 6 10e4 1 10e3 7 10e2 6 10e1 1 10e0 plus 6 10e16 9 10e15 5 10e14 8 10e13 3 10e12 0 10e11 5 10e10 2 10e9 4 10e8 5 10e7 1 10e6 8 10e5 0 10e4 4 10e3 6 10e2 3 10e1 8 10e0?
Correct:   6 10e16 9 10e15 5 10e14 8 10e13 3 10e12 0 10e1

Validation: 0it [00:00, ?it/s]


Question: What is 6 10e21 4 10e20 6 10e19 9 10e18 0 10e17 4 10e16 6 10e15 3 10e14 1 10e13 0 10e12 0 10e11 6 10e10 8 10e9 3 10e8 5 10e7 7 10e6 1 10e5 2 10e4 6 10e3 4 10e2 4 10e1 1 10e0 plus 7 10e22 6 10e21 7 10e20 5 10e19 2 10e18 8 10e17 5 10e16 9 10e15 9 10e14 6 10e13 3 10e12 1 10e11 2 10e10 2 10e9 3 10e8 8 10e7 0 10e6 0 10e5 9 10e4 9 10e3 6 10e2 5 10e1 7 10e0?
Correct:   8 10e22 3 10e21 2 10e20 2 10e19 1 10e18 9 10e17 0 10e16 6 10e15 2 10e14 7 10e13 3 10e12 1 10e11 9 10e10 0 10e9 7 10e8 3 10e7 7 10e6 2 10e5 2 10e4 6 10e3 0 10e2 9 10e1 8 10e0
Predicted: b'8 10e22 3 10e21 2 10e20 2 10e19 1 10e18 9 10e17 0 10e16 6 10e15 2 10e14 7 10e13 3 10e12 1 10e11 9 10e10 0 10e9 7 10e8 3 10e7 7 10e6 2 10e5 2 10e4 6 10e3 0 10e2 9 10e1 8 10e0'
Exact? True

Question: What is 6 10e5 6 10e4 1 10e3 7 10e2 6 10e1 1 10e0 plus 6 10e16 9 10e15 5 10e14 8 10e13 3 10e12 0 10e11 5 10e10 2 10e9 4 10e8 5 10e7 1 10e6 8 10e5 0 10e4 4 10e3 6 10e2 3 10e1 8 10e0?
Correct:   6 10e16 9 10e15 5 10e14 8 10e13 3 10e12 0 10e1

Validation: 0it [00:00, ?it/s]


Question: What is 6 10e21 4 10e20 6 10e19 9 10e18 0 10e17 4 10e16 6 10e15 3 10e14 1 10e13 0 10e12 0 10e11 6 10e10 8 10e9 3 10e8 5 10e7 7 10e6 1 10e5 2 10e4 6 10e3 4 10e2 4 10e1 1 10e0 plus 7 10e22 6 10e21 7 10e20 5 10e19 2 10e18 8 10e17 5 10e16 9 10e15 9 10e14 6 10e13 3 10e12 1 10e11 2 10e10 2 10e9 3 10e8 8 10e7 0 10e6 0 10e5 9 10e4 9 10e3 6 10e2 5 10e1 7 10e0?
Correct:   8 10e22 3 10e21 2 10e20 2 10e19 1 10e18 9 10e17 0 10e16 6 10e15 2 10e14 7 10e13 3 10e12 1 10e11 9 10e10 0 10e9 7 10e8 3 10e7 7 10e6 2 10e5 2 10e4 6 10e3 0 10e2 9 10e1 8 10e0
Predicted: b'8 10e22 3 10e21 2 10e20 2 10e19 1 10e18 9 10e17 0 10e16 6 10e15 2 10e14 7 10e13 3 10e12 1 10e11 9 10e10 0 10e9 7 10e8 3 10e7 7 10e6 2 10e5 2 10e4 6 10e3 0 10e2 9 10e1 8 10e0'
Exact? True

Question: What is 6 10e5 6 10e4 1 10e3 7 10e2 6 10e1 1 10e0 plus 6 10e16 9 10e15 5 10e14 8 10e13 3 10e12 0 10e11 5 10e10 2 10e9 4 10e8 5 10e7 1 10e6 8 10e5 0 10e4 4 10e3 6 10e2 3 10e1 8 10e0?
Correct:   6 10e16 9 10e15 5 10e14 8 10e13 3 10e12 0 10e1